In [2]:
pip install pyenchant nltk

# Projet L3 MIASHS extraction d'information
## A la poursuite de Satoshi Nakamoto

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/77/Satoshi_Nakamoto.jpg/1024px-Satoshi_Nakamoto.jpg" alt="Alt text" width="234" height="265">

**Satoshi Nakamoto** est le pseudonyme du créateur du Bitcoin, une innovation majeure dans le domaine des cryptomonnaies et des systèmes décentralisés. Entre 2008 et 2011, il a communiqué à travers des forums et des emails pour expliquer son projet, répondre aux questions et discuter des aspects techniques du Bitcoin. Cependant, son identité reste un mystère, et ses messages sont une source précieuse d’informations pour mieux comprendre ses idées, ses intentions et son évolution au fil du temps.

Dans ce projet, nous allons analyser les messages de Satoshi Nakamoto en utilisant Python, Pandas, beautiful soup et Power BI. L’objectif est d’extraire, organiser et visualiser différentes informations à partir de ces messages afin d’obtenir des indices sur son style d’écriture, ses sujets de discussion et son évolution au fil du temps.

### Objectifs du projet
À partir des messages de Satoshi Nakamoto extrait du site https://nakamotoinstitute.org/

- Collecter et structurer les données via du web scraping
- Mettre en forme les informations sous la forme d'un ou de plusieurs dataframes python
- Présenter les résultats sous forme de visualisations interactives avec Power BI

À travers ce projet, les étudiants appliqueront des concepts essentiels de l’analyse de données tout en explorant l’un des plus grands mystères du monde numérique : qui est Satoshi Nakamoto ?
Pour ce projet vous aurez besoins des packages suivants
- pyenchant
- nltk


### Rendu du projet :
Le projet se fait en binôme. Le rendu sera constitué de :
- Notebook python pour l'extraction des données
- une soutenance pour la présentation des résultats sous PowerBI

# Partie 1 extraction des premières informations
Dans un premier temps nous allons analyser la dynamique temporelle des échanges. En partant du premier thread à la page "https://satoshi.nakamotoinstitute.org/posts/p2pfoundation/1/" extraire les infos suivantes :
- Le titre du message
- La date du message en texte.

Vous pouvez analyser la date grâce aux informations suivantes.

In [3]:
import pandas as pd

# Example UTC datetime string
utc_string = "2009-02-11T22:27:00.000Z"
# Convert to datetime object
dt = pd.to_datetime(utc_string)

print(dt)
year = dt.year          # 2009
month = dt.month        # 2 (February)
day = dt.day            # 11
hour = dt.hour          # 22
minute = dt.minute      # 27
weekday = dt.day_name() # 'Wednesday'

print(year, month, day, hour, minute, weekday)

2009-02-11 22:27:00+00:00
2009 2 11 22 27 Wednesday


En regardant comment les adresses sont construites, faire de même pour les 10 premiers threads et mettre les résultats dans un dataframe python. Le sauvegarder et faire un premier tableau de bord powerBI avec ces informations.

Puis complèter les informations extraites en ajoutant les informations suivantes :
- Si c'est une réponse à un message. Si oui :
    - La date du premier message
    - Le nom de la personne à qui il répond
- Le nombre de mots du texte

Faites ces analyses sur l'ensemble des messages




In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
url = "https://satoshi.nakamotoinstitute.org/posts/p2pfoundation/1/"
def get_html(url):
  # récupérer le contenu HTML de la page
  response = requests.get(url)
  print(response.text)
  soup = BeautifulSoup(response.text, "html.parser")



*   **Recuperer le TITRE**





In [6]:
def get_title(soup):
    header = soup.find("div", class_="bg-dandelion")
    titre= header.find("div",class_="font-bold").text
    return titre



*   **Recuperer la DATE**





In [7]:
def get_date(soup):
    header = soup.find("div", class_="bg-dandelion")
    Date = header.find("time")['datetime']
     # Convert to datetime object
    dt = pd.to_datetime(Date)
    year = dt.year
    month = dt.month
    day = dt.day
    hour = dt.hour
    minute = dt.minute
    weekday = dt.day_name()

    return year,month,day,hour,minute,weekday



*   **Récuperer le message**


In [8]:
def get_message(soup):
    reponse_section = soup.find('div', class_='reply-to')
    reponse_auteur = "Non spécifié"
    reponse_date = "Inconnue"
    if reponse_section:
        reponse_auteur = reponse_section.find('strong').text.strip() if reponse_section.find('strong') else "Inconnu"
        reponse_date_tag = reponse_section.find('time')
        reponse_date = reponse_date_tag.text.strip() if reponse_date_tag else "Inconnue"

    # Si c'est une réponse et que ce n'est pas Satoshi, ne pas analyser
    if reponse_auteur != "Satoshi Nakamoto" and reponse_auteur != "Non spécifié":
        messages = ""
    else :
        find_messages = soup.find_all('div', class_='post')
        for message in find_messages:
          messages = message.get_text(strip=True)
          messages = messages.replace(";", " ")
    return messages



*   **Nombre de mots dans le texte**







In [9]:
def get_nb_mots(soup):
      messages =  get_message(soup)
      nb_mots = len(messages.split())
      return nb_mots

In [10]:
url = "https://satoshi.nakamotoinstitute.org/posts/p2pfoundation/1/"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
message = get_message(soup)
get_nb_mots(soup)

497

In [11]:
def reponse(titre):
  if titre.startswith("Re") or titre.startswith("Repost"):
    return 1
  else:
    return 0

# **10 PREMIERES PAGES**

In [35]:

from tabulate import tabulate
def get_info_posts(url, n):

    response = requests.get(url)
    forum_posts = BeautifulSoup(response.text, "html.parser")
    posts = forum_posts.find_all("article")

    urls = []
    for post in posts:
        # Récupère le lien
        lien = post.find("a")["href"]
        urls.append(lien)
    print("Nombre total de lien : ", len(urls))

    # Création d'une liste pour stocker les résultats de chaque post
    resultats = []

    for i in range(0, n):
        if n >= len(urls):
            n = len(urls)
        post_url = urls[i]
        post_response = requests.get(post_url)
        soup = BeautifulSoup(post_response.text, "html.parser")
        header = soup.find("div", class_="bg-dandelion")
        if header:
            titre = get_title(soup)
            messages = get_message(soup)
            Sentiment_neg,Sentiment_pos,Sentiment_neu,Sentiment_compound = get_sentiment(messages)
            year, month, day, hour, minute, weekday = get_date(soup)
            nb_mots = get_nb_mots(soup)
            langue = get_language(messages)
            Rep = reponse(titre)
            mot_frequent_1,freq_dist_1,mot_frequent_2,freq_dist_2,mot_frequent_3,freq_dist_3,mot_frequent_4,freq_dist_4,mot_frequent_5,freq_dist_5 = get_mot_frequents(messages)
            mots_rares_1,mots_rares_2,mots_rares_3,mots_rares_4,mots_rares_5 = get_mots_rares(messages)
            categorie = classer_message(messages)
            # Ajouter les résultats dans la liste
            resultats.append({
                #'Titre': titre,
                'Année': year,
                'Mois': month,
                'Jour': day,
                'Heure': hour,
                'Minute': minute,
                'Jour Semaine': weekday,
                'langue': langue,
                'Nombre de mots du message': nb_mots,
                'Reponse ?': Rep,
                'Mot_frequent_1': mot_frequent_1,
                'Freq_dist_1': freq_dist_1,
                'Mot_frequent_2': mot_frequent_2,
                'Freq_dist_2': freq_dist_2,
                'Mot_frequent_3': mot_frequent_3,
                'Freq_dist_3': freq_dist_3,
                'Mot_frequent_4': mot_frequent_4,
                'Freq_dist_4': freq_dist_4,
                'Mot_frequent_5': mot_frequent_5,
                'Freq_dist_5': freq_dist_5,
                'Mots_rares_1': mots_rares_1,
                'Mots_rares_2': mots_rares_2,
                'Mots_rares_3': mots_rares_3,
                'Mots_rares_4': mots_rares_4,
                'Mots_rares_5': mots_rares_5,
                'Catégorie': categorie,
                'Sentiment_neg': Sentiment_neg,
                'Sentiment_pos': Sentiment_pos,
                'Sentiment_neu': Sentiment_neu,
                'Sentiment_global': Sentiment_compound,
                #'Message': messages
            })

    # Convertir la liste de dictionnaires en un DataFrame
    df = pd.DataFrame(resultats)

    # Réinitialiser l'index pour commencer à 1
    df.index = df.index + 1

    # Nommer la colonne d'index
    df.index.name = 'Numéro de Post'
    get_temps_moyen_reponse(df)

    # Affichage en tableau bien ordonné
    print(tabulate(df, headers='keys', tablefmt='fancy_grid'))

    # Exporter au format CSV
    df.to_csv("data_Satoshi_posts.csv", index=False, sep=";", decimal=",")


In [36]:
get_info_posts("https://satoshi.nakamotoinstitute.org/posts/", 543)

Nombre total de lien :  543
Temps moyen de réponse : 1035.51 minutes

Résumé des temps de réponse :
+---+--------------------+-------------------+-----------------+-----------------+
|   | Nombre de réponses | Temps moyen (min) | Temps max (min) | Temps min (min) |
+---+--------------------+-------------------+-----------------+-----------------+
| 0 |       487.0        |      1035.51      |     76499.0     |       1.0       |
+---+--------------------+-------------------+-----------------+-----------------+
╒══════════════════╤═════════╤════════╤════════╤═════════╤══════════╤════════════════╤═════════════════╤═════════════════════════════╤═════════════╤══════════════════╤═══════════════╤══════════════════╤═══════════════╤══════════════════╤═══════════════╤════════════════════╤═══════════════╤════════════════════════╤═══════════════╤════════════════╤════════════════════╤════════════════╤══════════════════╤════════════════╤═════════════╤═════════════════╤═════════════════╤═════════════

In [ ]:
# c'est un peu de la triche mais ça marche lol
#def get_info(n):
 # for i in range(1,n):
  #    url = f"https://satoshi.nakamotoinstitute.org/posts/p2pfoundation/{i}/"
    #  response = requests.get(url)

    #  if response.status_code != 200:
     #     url = f"https://satoshi.nakamotoinstitute.org/posts/bitcointalk/{i}/"
     #     response = requests.get(url)

      #soup = BeautifulSoup(response.text, "html.parser")
      #header = soup.find("div", class_="bg-dandelion")

      #if header:
        #  titre_= header.find("div", class_="font-bold")
        #  titre = titre.text if titre else "Titre non trouvé"

         # date = header.find("time")
         # date = date['datetime'] if date else "Date non trouvée"

          #print(titre, date)
     # else:
       #   print(f"Aucun header trouvé pour {url}")
#get_info(11)

# Partie 2 analyse du texte
On veut maintenant analyser le contenu des messages. Pour cela on peut utiliser NLTK pour analyser les sentiments de la façon suivante :

In [14]:
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

# Download VADER if not already downloaded
nltk.download('vader_lexicon')

# Initialize sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Example message from Satoshi
message = "Bitcoin is an electronic cash system that uses a peer-to-peer network to prevent double-spending."

# Analyze sentiment
sentiment = sia.polarity_scores(message)

# Print results
print(sentiment)
# {'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'compound': 0.4215}

{'neg': 0.0, 'neu': 0.916, 'pos': 0.084, 'compound': 0.0258}


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [15]:
def get_sentiment(message):
  Sentiment_neg = sia.polarity_scores(message)['neg']
  Sentiment_pos = sia.polarity_scores(message)['pos']
  Sentiment_neu = sia.polarity_scores(message)['neu']
  Sentiment_compound = sia.polarity_scores(message)['compound']
  return Sentiment_neg,Sentiment_pos,Sentiment_neu,Sentiment_compound

On peut aussi utiliser pyenchant pour detecter si le texte est écrit en anglais ou en américain. Par exemple :

In [16]:
!apt-get update
!apt-get install -y libenchant1c2a
!apt-get update
!apt-get install -y libenchant-2-dev
import enchant

# Create spell-checker dictionaries for US and UK English
us_dict = enchant.Dict("en_US")
uk_dict = enchant.Dict("en_GB")
message1 = "We should optimise the colour of this interface."  # British spelling
words = message1.split()
us_count = sum(1 for word in words if us_dict.check(word) and not uk_dict.check(word))
uk_count = sum(1 for word in words if uk_dict.check(word) and not us_dict.check(word))
if us_count > uk_count:
         ("American English")
elif uk_count > us_count:
        print("British English")
else:
        print("Unclear/Mixed")

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,383 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,810 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,000 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com

In [17]:
def get_language(message):
  us_dict = enchant.Dict("en_US")
  uk_dict = enchant.Dict("en_GB")
  rep=""
  message1 = "We should optimise the colour of this interface."  # British spelling
  words = message1.split()
  us_count = sum(1 for word in words if us_dict.check(word) and not uk_dict.check(word))
  uk_count = sum(1 for word in words if uk_dict.check(word) and not us_dict.check(word))
  if us_count > uk_count:
          rep = "American English"
  elif uk_count > us_count:
          rep="British English"
  else:
          rep = "Unclear/Mixed"
  return rep


En utilisant ces deux biliothèques, enrichir la base de données en analysant chaque texte. Attention, quand le texte est une réponse, il faut analyser seulement le texte de Satoshi.

# Partie 3 enrichissement

Vous devez maintenant complèter votre analyse en ajoutant le plus d'informations possibles. Vous pouvez par exemple :
- Traiter les mails de Satoshi
- Traiter le code produit par Satoshi
- Chercher les mentions à différentes personnes
- Faire des analyses des mots qui reviennent le plus souvent
- classement des messages par thèmes (technique, politique, autres ...)
- Analyse du temps de réponse moyen
- Analyses des principaux interlocuteurs de Satoshi
- Analyse du style
- Chercher les mots unique ou rare -vous pouvez utiliser `nltk.FreqDist`
- ect ...

Stocker toutes ces informations dans une ou plusieurs dataframe

# **1. ANALYSE  DES MAILS**





In [ ]:
get_html("https://satoshi.nakamotoinstitute.org/emails/")

In [ ]:
get_html("https://satoshi.nakamotoinstitute.org/emails/cryptography/1/")

In [18]:
def get_contenu_brut(soup):
    contenu = soup.get_text(separator="\n", strip=True)
    if contenu:
        return contenu
    else:
        return ""


In [ ]:
def get_message_mail(soup):
    # Récupérer tout le texte de la page
    full_text = soup.get_text(separator="\n", strip=True)

    # Diviser le texte en lignes
    lines = full_text.split("\n")

    # Filtrer pour ne garder que les lignes pertinentes, en excluant celles qui commencent par ">"
    message_lines = []
    for line in lines:
        if not line.startswith(">") and line.strip():  # Ignorer les citations (qui commencent par ">") et les lignes vides
            message_lines.append(line.strip())

    # Rejoindre les lignes pertinentes pour obtenir le message final
    message = "\n".join(message_lines)

    return message


In [19]:
def reponse_mail(contenu):
    if not contenu:
        return False
    if "> " in contenu:
        return True
    else:
        return False


In [20]:
def get_nb_mots_mail(soup):
      messages =  get_message_mail(soup)
      nb_mots = len(messages.split())
      return nb_mots

In [42]:
from tabulate import tabulate
def get_info_mails(url, n):

    response = requests.get(url)
    if response.status_code == 200:
        Emails = BeautifulSoup(response.text, "html.parser")
        mails = Emails.find_all("article")

    urls = []
    for mail in mails:
        # Récupère le lien
        lien = mail.find("a")["href"]
        urls.append(lien)
    print("Nombre total de lien : ", len(urls))

    # Création d'une liste pour stocker les résultats de chaque post
    resultats = []

    for i in range(0, n):
        if n >= len(urls):
            n = len(urls)
        post_url = urls[i]
        post_response = requests.get(post_url)
        soup = BeautifulSoup(post_response.text, "html.parser")
        header = soup.find("div", class_="bg-dandelion")
        if header:
            #titre = get_title(soup)
            #messages = get_message_mail(soup)
            contenu = get_contenu_brut(soup)
            #Sentiment_neg,Sentiment_pos,Sentiment_neu,Sentiment_compound = get_sentiment(messages)
            year, month, day, hour, minute, weekday = get_date(soup)
            #nb_mots = get_nb_mots_mail(soup)
            #langue = get_language(messages)
            Rep = reponse_mail(contenu)
            #mot_frequents= get_mot_frequents(messages)
            # Ajouter les résultats dans la liste
            resultats.append({
                #'Titre': titre,
                'Année': year,
                'Mois': month,
                'Jour': day,
                'Heure': hour,
                'Minute': minute,
                'Jour Semaine': weekday,
                #'langue': langue,
                #'Nombre de mots du message': nb_mots,
                'Reponse ?': Rep,
                #'Mots frequents': mot_frequents,
                #'Sentiment_neg': Sentiment_neg,
                #'Sentiment_pos': Sentiment_pos,
                #'Sentiment_neu': Sentiment_neu,
                #'Sentiment_global': Sentiment_compound,
                #'Message': messages
            })

    # Convertir la liste de dictionnaires en un DataFrame
    df = pd.DataFrame(resultats)

    # Réinitialiser l'index pour commencer à 1
    df.index = df.index + 1

    # Nommer la colonne d'index
    df.index.name = 'Numéro de mail'

    get_temps_moyen_reponse(df)
    # Affichage en tableau bien ordonné
    print(tabulate(df, headers='keys', tablefmt='fancy_grid'))

    # Exporter au format CSV
    df.to_csv("data_Satoshi_mail.csv", index=False, sep=";", decimal=",")

In [43]:
get_info_mails("https://satoshi.nakamotoinstitute.org/emails/", 39)

Nombre total de lien :  39
Temps moyen de réponse : 20053.60 minutes

Résumé des temps de réponse :
+---+--------------------+-------------------+-----------------+-----------------+
|   | Nombre de réponses | Temps moyen (min) | Temps max (min) | Temps min (min) |
+---+--------------------+-------------------+-----------------+-----------------+
| 0 |        20.0        |      20053.6      |    335938.0     |      58.0       |
+---+--------------------+-------------------+-----------------+-----------------+
╒══════════════════╤═════════╤════════╤════════╤═════════╤══════════╤════════════════╤═════════════╤═════════════════════╕
│   Numéro de mail │   Année │   Mois │   Jour │   Heure │   Minute │ Jour Semaine   │ Reponse ?   │ datetime            │
╞══════════════════╪═════════╪════════╪════════╪═════════╪══════════╪════════════════╪═════════════╪═════════════════════╡
│                1 │    2008 │     10 │     31 │      18 │       10 │ Friday         │ False       │ 2008-10-31 18:1

In [32]:
import nltk
from collections import Counter

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [33]:
def get_mot_frequents(texte):
    # Tokenisation
    mots = nltk.word_tokenize(texte.lower())
    # Garde seulement les mots alphabétiques
    mots = [mot for mot in mots if mot.isalpha()]

    # Calcul de la fréquence
    freq_dist = Counter(mots)
    mots_frequents = freq_dist.most_common(5)

    # Gestion du cas où il y a moins de 5 mots fréquents
    mot_1, freq_1, mot_2, freq_2, mot_3, freq_3, mot_4, freq_4, mot_5, freq_5 = "N/A", 0, "N/A", 0, "N/A", 0, "N/A", 0, "N/A", 0  # Valeurs par défaut

    for i, (mot, freq) in enumerate(mots_frequents):
        if i == 0:
            mot_1, freq_1 = mot, freq
        elif i == 1:
            mot_2, freq_2 = mot, freq
        elif i == 2:
            mot_3, freq_3 = mot, freq
        elif i == 3:
            mot_4, freq_4 = mot, freq
        elif i == 4:
            mot_5, freq_5 = mot, freq

    return mot_1, freq_1, mot_2, freq_2, mot_3, freq_3, mot_4, freq_4, mot_5, freq_5

In [23]:
from nltk import FreqDist

def get_mots_rares(messages):

    mots = [mot for mot in messages.lower().split() if mot.isalpha()]

    # Calcule la frequence des mots
    freq_dist = FreqDist(mots)

    # mot avec freq =  1
    rares = [mot for mot, freq in freq_dist.items() if freq == 1]

    # Gestion du cas où il y a moins de 5 mots rares
    rares_1, rares_2, rares_3, rares_4, rares_5 = "N/A", "N/A", "N/A", "N/A", "N/A"  # Valeurs par défaut

    # Affecter les valeurs des mots rares si disponibles
    if len(rares) > 0:
        rares_1 = rares[0]
    if len(rares) > 1:
        rares_2 = rares[1]
    if len(rares) > 2:
        rares_3 = rares[2]
    if len(rares) > 3:
        rares_4 = rares[3]
    if len(rares) > 4:
        rares_5 = rares[4]

    return rares_1, rares_2, rares_3, rares_4, rares_5

**Classement par thème**

In [24]:
from collections import defaultdict
import re

thematiques = {
    "technique": [
        "bitcoin", "blockchain", "network", "encryption", "hash", "node", "transaction",
        "proof", "mining", "signature", "protocol", "merkle", "wallet", "software", "bytes"
    ],
    "économie": [
        "money", "currency", "inflation", "deflation", "value", "price", "market",
        "capital", "supply", "demand", "monetary", "scarcity"
    ],
    "politique": [
        "government", "regulation", "freedom", "privacy", "censorship", "control",
        "authority", "liberty", "centralized", "decentralized"
    ],
    "communauté": [
        "user", "community", "people", "idea", "discussion", "opinion", "developer",
        "adoption", "feedback", "contribution", "support"
    ],
    "sécurité": [
        "attack", "exploit", "vulnerability", "secure", "security", "risk", "defense",
        "malicious", "threat", "protection"
    ],
    "juridique": [
        "law", "legal", "tax", "jurisdiction", "regulatory", "compliance", "framework"
    ]
}


def classer_message(message_texte):
    # Nettoyage basique du texte et tokenisation
    tokens = re.findall(r'\b\w+\b', message_texte.lower())
    compteur = defaultdict(int)
    for mot in tokens:
        for theme, mots_clefs in thematiques.items():
            if mot in mots_clefs:
                compteur[theme] += 1
    if compteur:
        return max(compteur, key=compteur.get)
    else:
        return "autres"



**Temps moyen de reponse**

In [44]:
def get_temps_moyen_reponse(df):
    # Conversion des colonnes temporelles
    dates = df[["Année", "Mois", "Jour", "Heure", "Minute"]].copy()
    dates.columns = ["year", "month", "day", "hour", "minute"]
    df["datetime"] = pd.to_datetime(dates)

    # S'assurer que la colonne "Reponse ?" est bien booléenne
    df["Reponse ?"] = df["Reponse ?"].astype(bool)
    # Trier par date (par sécurité)
    df = df.sort_values("datetime").reset_index(drop=True)

    # Calcul des écarts de temps
    ecarts = []
    for i in range(1, len(df)):
        if df.loc[i, "Reponse ?"]:
            t1 = df.loc[i - 1, "datetime"]
            t2 = df.loc[i, "datetime"]
            ecart = (t2 - t1).total_seconds() / 60  # minutes
            ecarts.append(ecart)

    if ecarts:
        moyenne_minutes = sum(ecarts) / len(ecarts)
        print(f"Temps moyen de réponse : {moyenne_minutes:.2f} minutes")
        # Tableau récapitulatif
        df_resume = pd.DataFrame({
            "Nombre de réponses": [len(ecarts)],
            "Temps moyen (min)": [round(moyenne_minutes, 2)],
            "Temps max (min)": [round(max(ecarts), 2)],
            "Temps min (min)": [round(min(ecarts), 2)]
        })
        print("\nRésumé des temps de réponse :")
        print(tabulate(df_resume, headers='keys', tablefmt='pretty'))
        df_resume.to_csv("frequence_reponse.csv", index=False, sep=";", decimal=",")
    else:
        print("Aucune réponse trouvée dans les données.")
    return ecarts


# Partie 4 : Tableau de bord
Vous devez maintenant utiliser powerBI pour analyser les dataframe que vous avez produit. Utiliser PowerBI pour réprésenter :
- La fréquence d'activité de Satoshi suivant plusieurs critère
- L'évolution de son activité dans le temps
- Les interactions avec la communauté
- La représentation des différentes information que vous avez extraite (le style, les thèmes principaux de discussion, ect )

Essayer d'en déduire un maximum d'informations sur Satoshi